In [1]:
import os
import pickle

data_folder = '/home/mcwave/code/automath/atp/datasets/provability/mathlib4_states_w_proof/'
file_names = os.listdir(data_folder)

data = []

count = 0
for file_name in file_names:
    if not file_name.endswith("pkl"):
        continue
    if not 'Algebra' in file_name:
        continue
    count += 1
    if count <= 5:
        continue
    print("Loading", file_name)
    file_path = os.path.join(data_folder, file_name)
    fin = open(file_path, 'rb')
    while True:
        try:
            pair = pickle.load(fin)
            data.append(pair) #(pair[1][0], pair[1][2][0]))
        except:
            break
    break

print(len(data), "examples loaded")

Loading Mathlib__Algebra__Ring__Subring__Pointwise.lean.pkl
12680 examples loaded


In [2]:
from utils.lean_math_utils import *
from utils.lean_theorem_utils import *

def count_lines(string):
    # Split the string into lines
    lines = string.splitlines()
    # Count the number of lines
    return len(lines)

def extract_first_case(state_pp):
    state_pp = state_pp.strip()
    if not state_pp.startswith('case'):
        return state_pp
    lines = state_pp.split('\n')
    first_case = []
    for line in lines[1:]:
        if line.strip().startswith('case'):
            break
        if line.strip() != '':
            first_case.append(line)
    return '\n'.join(first_case)


# Params:
#   hyp: tuple(name, type)
#   tactics: list(tactic)
def is_hypothesis_useful(hyp, tactics):
    for tactic in tactics:
        tokens = tokenize_lean_tactic(tactic)
        if hyp[0] in tokens:
            idx = tokens.index(hyp[0])
            if idx > 0:
                if hyp[0].startswith('h'):
                    return True
                if tokens[idx - 1] == 'exact':
                    return True
                if tokens[idx - 1] == 'at':
                    return True
                if tokens[idx - 1] == '[':
                    return True
                if idx < len(tokens) - 1 and tokens[idx + 1] == ']':
                    return True
                for operator in TargetNode.operators:
                    if operator in hyp[1]:
                        return True
    return False

def create_hypothesis_predict_data(raw_state_pp, tactics, theorem_name):
    is_case = raw_state_pp.strip().startswith('case')
    state_pp = extract_first_case(raw_state_pp)
    if is_case and count_lines(state_pp) < count_lines(raw_state_pp) - 2:
        tactics = tactics[0:1]
    #
    premise = Premise()
    premise.theorem_name = theorem_name
    premise.parse_state(state_pp)
    #
    useful_hypotheses, useless_hypotheses = [], OrderedDict()
    for hyp in premise.hypotheses.items():
        useful = is_hypothesis_useful(hyp, tactics)
        if useful:
            #print("YES:", hyp)
            useful_hypotheses.append(hyp)
        else:
            #print("NO :", hyp)
            useless_hypotheses[hyp[0]] = hyp[1]
    premise.hypotheses = useless_hypotheses
    return premise, useful_hypotheses

idx = 120

state_pp = data[idx][1][0]
tactics = data[idx][1][2]
theorem_name = data[idx][0][3]

print("STATE_PP:\n" + state_pp)

print("TACTICS:\n" + "\n".join(tactics))

premise, useful_hypotheses = create_hypothesis_predict_data(state_pp, tactics, theorem_name)

print("STATE_PP:\n" + premise.to_theorem_code())
print("\nHYPOTHESES:\n", useful_hypotheses)


STATE_PP:
case h
M : Type u_1
R : Type u_2
inst✝² : Monoid M
inst✝¹ : Ring R
inst✝ : MulSemiringAction M R
a : M
S : Subring R
nvar0 : R
⊢ nvar0 ∈ map (MulSemiringAction.toRingHom M R a) S ↔ nvar0 ∈ a • S
TACTICS:
symm
simp [eq_comm (a := a)]
cases S
rw [smul_neg]
rw [← eq_f₀']
STATE_PP:
theorem Subring.pointwise_smul_def (M: Type u_1) (R: Type u_2) (inst✝²: Monoid M) (inst✝¹: Ring R) (inst✝: MulSemiringAction M R) (a: M) (S: Subring R) (nvar0: R) : nvar0 ∈ map (MulSemiringAction.toRingHom M R a) S ↔ nvar0 ∈ a • S :=

HYPOTHESES:
 []


In [3]:
from datasets import Dataset

MIN_LENGTH = 4

state_pps = []
target_hyps = []
seen_hashes = set()
fin = open('/home/mcwave/code/axiomatization/datasets/mathlib4_all_states_w_proof_hyp_pred.pkl', 'rb')

while True:
    try:
        premise, hypotheses = pickle.load(fin)
        state_pp = premise.to_theorem_code()
        target_hyp = str([x[1] for x in hypotheses])
        hash_value = hash(state_pp + '|' + target_hyp)
        if hash_value in seen_hashes:
            continue
        else:
            seen_hashes.add(hash_value)
        #data.append((state_pp, target_hyp))
        if len(state_pp) < 4 or len(target_hyp) < 4:
            continue
        state_pps.append(state_pp)
        target_hyps.append(target_hyp)
    except:
        break
    
fin.close()

print(len(state_pps))

3420301


In [3]:
import torch
from transformers import AutoTokenizer
from datasets import Dataset

MAX_LENGTH = 300

# Initialize tokenizer and model

model_name = "morph-labs/morph-prover-v0-7b" #"internlm/internlm2-math-7b" #"ScalableMath/Lean-STaR-plus"  # 'Saisam/gpt-neo-math-small' #

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.padding_side = "right"

# Define the separator token
sep_token = "<sep>"
pad_token = "<pad>"

# Check if the separator token already exists in the vocabulary
if sep_token not in tokenizer.get_vocab():
    tokenizer.add_tokens([sep_token])
if pad_token not in tokenizer.get_vocab():
    tokenizer.add_tokens([pad_token])

# Set the separator token
tokenizer.sep_token = sep_token
tokenizer.pad_token = pad_token

tokenizer.add_special_tokens({
    'sep_token': sep_token,
    'pad_token': pad_token
})

# tokenizer = AutoTokenizer.from_pretrained("datasets/text_for_tokenization/mathlib4_20240617_bpe_tokenizer")
# # Define the tokens
# sep_token = "<sep>"
# pad_token = "<pad>"

# # Set the sep_token and pad_token
# tokenizer.sep_token = sep_token
# tokenizer.pad_token = pad_token
# #tokenizer.add_special_tokens({'sep_token': '[SEP]'})


# Function to tokenize and prepare the data
def prepare_data(examples):
    # Concatenate instruction and response with a separator
    full_texts = [f"{instruction} <sep> {response}" for instruction, response in zip(examples['instruction'], examples['response'])]
    
    # Tokenize the full texts
    encodings = tokenizer(full_texts, truncation=True, padding='max_length', max_length=MAX_LENGTH, return_tensors='pt')
    #print(encodings)
    
    # Create attention masks: 1 for response tokens, 0 for instruction tokens and padding
    attention_masks = []
    labels = []
    
    for input_ids in encodings['input_ids']:
        attention_mask = torch.zeros_like(input_ids)
        label = input_ids.clone()
        
        pad_token_idx = (input_ids == tokenizer.pad_token_id).nonzero()
        end_idx = pad_token_idx[0].item() if len(pad_token_idx) > 0 else len(input_ids)
        sep_token_idx = (input_ids == tokenizer.sep_token_id).nonzero()
        #print("sep_token_idx:", sep_token_idx)
        if len(sep_token_idx) == 0:
            sep_token_idx = 0
        else:
            sep_token_idx = sep_token_idx.item()

        attention_mask[0:end_idx] = 1
        attention_masks.append(attention_mask)
        
        label[0:sep_token_idx+1] = -100
        labels.append(label)
    
    return {
        'input_ids': encodings['input_ids'],
        'attention_mask': torch.stack(attention_masks),
        'labels': torch.stack(labels)
    }

# Assuming you have two lists: instructions and responses
instructions = state_pps #["Instruction 1", "Instruction 2"]
responses = target_hyps #["Response 1", "Response 2"]
#instructions = ["Instruction 1", "Instruction 2"]
#responses = ["Response 1", "Response 2"]


# Create the Hugging Face dataset
dataset = Dataset.from_dict({
    'instruction': instructions,
    'response': responses
})

# Apply the tokenization and preparation function
tokenized_dataset = dataset.map(
    prepare_data,
    batched=True,
    num_proc=4
    #remove_columns=dataset.column_names
)

NameError: name 'state_pps' is not defined

In [4]:
import torch
from transformers import AutoTokenizer
from datasets import load_dataset, Dataset, load_from_disk

#tmp = tokenized_dataset.shuffle(seed=42).train_test_split(test_size=0.007, seed=42)
#tokenized_train = tmp['train']
#tokenized_test = tmp['test']

#tokenized_train.save_to_disk('datasets/predict_hyp_tokenized_train.dataset')
#tokenized_test.save_to_disk('datasets/predict_hyp_tokenized_test.dataset')

tokenized_train = load_from_disk('datasets/predict_hyp_tokenized_train.dataset')
tokenized_test = load_from_disk('datasets/predict_hyp_tokenized_test.dataset')

Loading dataset from disk:   0%|          | 0/30 [00:00<?, ?it/s]

In [12]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from trl import setup_chat_format
 
# Hugging Face model id
model_id = "codellama/CodeLlama-7b-hf" # or `mistralai/Mistral-7B-v0.1`
 
# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)
 
# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    attn_implementation="flash_attention_2",
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Embedding(32002, 2304, padding_idx=0)

In [15]:
from transformers import Trainer, TrainingArguments
from datasets import load_dataset,load_metric
from transformers import AutoTokenizer, DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir="datasets/predict-hyp-gemma-2b",
    evaluation_strategy="steps", #"epochs"
    learning_rate=2e-5,  # PAY ATTENTION TO LEARNING RATE!
    weight_decay=0.01,
    per_device_train_batch_size=12,
    per_device_eval_batch_size=2,
    num_train_epochs=4,
    bf16=True,
    max_grad_norm=1.0,
    save_steps=20000,
    eval_steps=20000,
    logging_steps=20000,
    save_total_limit=3,
    #load_best_model_at_end=True,
    push_to_hub=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    data_collator=data_collator,
)

cp_path = 'datasets/predict-hyp-gemma2-2b/checkpoint-40000'

trainer.train()

/home/mcwave/anaconda3/envs/axiom/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


RuntimeError: value cannot be converted to type at::BFloat16 without overflow

In [22]:
tokenizer.decode(tokenized_test[0]['input_ids'])

"<s> theorem CategoryTheory.mop_rightUnitor (C: Type u₁) (inst✝¹: Category.{v₁, u₁} C) (inst✝: MonoidalCategory C) (X: C) (⊢ ¬¬¬¬(ρ_ X).mop λ_ { unmop: = X }) :  := <sep>  ['¬¬¬(ρ_ X).mop = λ_ { unmop := X }', '= X }']<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><p

In [37]:
test_case

{'instruction': 'theorem Equiv.embeddingFinSucc_fst (m n✝ n: ℕ) (ι: Type u_1) : ⇑((embeddingFinSucc n ι) e).fst = ⇑e ∘ Fin.succ :=',
 'response': "['Fin (n + 1) ↪ ι']",
 'input_ids': [1,
  16980,
  8391,
  449,
  28723,
  18320,
  3202,
  11491,
  5173,
  588,
  28730,
  28722,
  303,
  325,
  28719,
  307,
  229,
  159,
  160,
  307,
  28747,
  28705,
  229,
  135,
  152,
  28731,
  325,
  28980,
  28747,
  5707,
  332,
  28730,
  28740,
  28731,
  714,
  28705,
  229,
  138,
  148,
  1880,
  18320,
  3202,
  11491,
  5173,
  588,
  307,
  28705,
  28980,
  28731,
  317,
  609,
  28722,
  303,
  327,
  28705,
  229,
  138,
  148,
  28706,
  28705,
  229,
  139,
  155,
  3727,
  28723,
  2913,
  588,
  2137,
  28705,
  32000,
  28705,
  5936,
  11491,
  325,
  28711,
  648,
  28705,
  28740,
  28731,
  28705,
  229,
  137,
  173,
  28705,
  28980,
  1421,
  32001,
  32001,
  32001,
  32001,
  32001,
  32001,
  32001,
  32001,
  32001,
  32001,
  32001,
  32001,
  32001,
  32001,
  3200

In [42]:
test_case = tokenized_test[7]
#input_ids = torch.tensor(test_case['input_ids']).unsqueeze(0).to('cuda')  # Add batch dimension
input_ids = tokenizer.encode(test_case['instruction'], return_tensors='pt').to('cuda')
print(input_ids)
print("inputs:", tokenizer.decode(input_ids[0], skip_special_tokens=True))
labels = [x for x in test_case['labels'] if x >= 0]
labels = torch.tensor(labels).to('cuda')
print("labels:", tokenizer.decode(labels, skip_special_tokens=True))

# Generate output
with torch.no_grad():
    outputs = model.generate(input_ids, max_new_tokens=50) #max_length=MAX_LENGTH)

print(outputs)
    
# Decode the generated output and the true labels
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
true_text = tokenizer.decode(labels, skip_special_tokens=True)

# Compare the results
print("Generated:", generated_text)
print("True:", true_text)

tensor([[    1, 16980,  8391,   449, 28723, 18320,  3202, 11491,  5173,   588,
         28730, 28722,   303,   325, 28719,   307,   229,   159,   160,   307,
         28747, 28705,   229,   135,   152, 28731,   325, 28980, 28747,  5707,
           332, 28730, 28740, 28731,   714, 28705,   229,   138,   148,  1880,
         18320,  3202, 11491,  5173,   588,   307, 28705, 28980, 28731,   317,
           609, 28722,   303,   327, 28705,   229,   138,   148, 28706, 28705,
           229,   139,   155,  3727, 28723,  2913,   588,  2137]],
       device='cuda:0')
inputs: theorem Equiv.embeddingFinSucc_fst (m n✝ n: ℕ) (ι: Type u_1) : ⇑((embeddingFinSucc n ι) e).fst = ⇑e ∘ Fin.succ :=
labels:  ['Fin (n + 1) ↪ ι']
tensor([[    1, 16980,  8391,   449, 28723, 18320,  3202, 11491,  5173,   588,
         28730, 28722,   303,   325, 28719,   307,   229,   159,   160,   307,
         28747, 28705,   229,   135,   152, 28731,   325, 28980, 28747,  5707,
           332, 28730, 28740, 28731,   714, 287